# Test db connection and populating a table

In [45]:
import pyodbc
import pandas as pd

server = "localhost,5432"
db_name = "ModMon"
driver = "/usr/local/lib/psqlodbcw.so"

cnxn = pyodbc.connect("DRIVER={" + driver + "};SERVER=" + server + ";DATABASE=" + db_name + ";Trusted_Connection=yes;")

In [46]:
pd.read_sql("SELECT * FROM information_schema.tables;", cnxn)['table_name']

0                      teams
1                     models
2                   projects
3              modelversions
4                   datasets
               ...          
191          foreign_servers
192       _pg_foreign_tables
193    foreign_table_options
194           foreign_tables
195        _pg_user_mappings
Name: table_name, Length: 196, dtype: object

In [47]:
pd.read_sql('SELECT * FROM teams;', cnxn)

,id,contact_name,contact_email


In [48]:
cursor = cnxn.cursor()

In [49]:
cursor.execute('''
                INSERT INTO teams (id, contact_name, contact_email)
                VALUES
                (1, 'Ed Chalstrey', 'echalstrey@turing.ac.uk'),
                (2, 'Jack Roberts', 'jroberts@turing.ac.uk')
                ''')

In [50]:
pd.read_sql('SELECT * FROM teams;', cnxn)

,id,contact_name,contact_email
0,1,Ed Chalstrey,echalstrey@turing.ac.uk
1,2,Jack Roberts,jroberts@turing.ac.uk


# Import data from `models/sklearn_basic` to the db

In [51]:
import json

In [52]:
# After running models/sklearn_basic/prediction-metrics.py
with open('../models/sklearn_basic/analyst_scripts/metrics.json') as json_file:
    metrics = json.load(json_file)
metrics

{'r2': 0.2732441488584465,
 'mse': 0.5535172595095682,
 'mar': 0.5846394599796879}

In [53]:
cursor.execute('''
                DROP TABLE IF EXISTS test;
                ''')

In [54]:
cursor.execute('''
                CREATE TABLE test (
                  "name" VARCHAR(10),
                  "value" float
                  );
                ''')

In [55]:
for name, value in metrics.items():
    cursor.execute('''
        INSERT INTO 
            test
            (name, value)
        VALUES
            (?, ?)
    ''', name, value)

In [56]:
pd.read_sql('SELECT * FROM test;', cnxn)

,name,value
0,r2,0.273244
1,mse,0.553517
2,mar,0.584639


In [57]:
cursor.execute('''DROP TABLE test;''')

In [58]:
cursor.execute('''
                INSERT INTO projects (id, name, description)
                VALUES
                (1, 'sklearn_basic', 'Test project with sci-kit-learn and Python');
                ''')

In [59]:
pd.read_sql('SELECT * FROM projects;', cnxn)

,id,name,description
0,1,sklearn_basic,Test project with sci-kit-learn and Python


In [60]:
cursor.execute('''delete from projects;''')

In [61]:
cnxn.commit()
cnxn.close()